In [5]:
import numpy as np
import albumentations as A      # Video manipulation
import cv2
from scapy.all import *       # PCAP manipulation
import random
import os

In [ ]:
# Video manipulation
### CHANGE THESE SETTINGS FOR NEW DIFFERENCES ###
video_aug = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.2),
])

def video_to_frames(video_path, grayscale=False):
    """
    Mainly for testing purposes
    Converts an MP4 video to a NumPy array of frames.
    
    Returns np.ndarray: Array of shape (N, H, W, C) [RGB] or (N, H, W) [Grayscale].
    """
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # OpenCV uses BGR, convert to RGB
        
        frames.append(frame)
    
    cap.release()
    return np.array(frames)


def augment_video(in_video_path, out_video_path, grayscale=False, fps=24):
    cap = cv2.VideoCapture(in_video_path)
    frames = []
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # print('w: ', frame_width, ' h: ', frame_height)
    out = cv2.VideoWriter(out_video_path, fourcc, fps, (frame_width, frame_height))
    
    while True and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)   # OpenCV uses BGR, convert to RGB
        
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print('Created new augmented video at: ', out_video_path)


In [28]:
# Video Manipulation Testing
VIDEO_OUTPUT_PATH_PREFIX = './augmented_data/video/'
VIDEO_INPUT_PATH_PREFIX = './Anusha Recordings/Anusha Recordings/'
VIDEO_SUFFIX = '.mp4'

video_path = VIDEO_INPUT_PATH_PREFIX + "A.mp4"
frames = video_to_frames(video_path, grayscale=False)  # Shape: (N, H, W, 3)
augmented = video_aug(images=frames)
augmented_video_frames = augmented['images']

print(f"Original shape: {frames.shape}")
print(f"Augmented shape: {augmented_video_frames.shape}")
# Example output (shape depends on pipeline): Augmented shape: (10, 224, 224, 3)

Original shape: (48, 360, 428, 3)
Augmented shape: (48, 256, 256, 3)


In [29]:
# Testing Video Augmentation
output_video_path = VIDEO_OUTPUT_PATH_PREFIX + 'A' + VIDEO_SUFFIX
augment_video(video_path, output_video_path)

w:  428  h:  360
Created new augmented video at:  ./augmented_data/video/A.mp4


In [ ]:
# loop through all og video files
pre_imgs = os.listdir(VIDEO_INPUT_PATH_PREFIX)
augmentation_type = '-test'
# print(pre_imgs)
for p in pre_imgs:
    p = p[:-4]
    augment_video(VIDEO_INPUT_PATH_PREFIX+p+VIDEO_SUFFIX, 
                  VIDEO_OUTPUT_PATH_PREFIX+p+augmentation_type+VIDEO_SUFFIX)

Created new augmented video at:  ./augmented_data/video/Which-test.mp4
Created new augmented video at:  ./augmented_data/video/How-test.mp4
Created new augmented video at:  ./augmented_data/video/A-test.mp4
Created new augmented video at:  ./augmented_data/video/Am-test.mp4
Created new augmented video at:  ./augmented_data/video/Mornin-test.mp4
Created new augmented video at:  ./augmented_data/video/My-test.mp4
Created new augmented video at:  ./augmented_data/video/Teammates-test.mp4
Created new augmented video at:  ./augmented_data/video/291c-test.mp4
Created new augmented video at:  ./augmented_data/video/.DS_S-test.mp4
Created new augmented video at:  ./augmented_data/video/Are-test.mp4
Created new augmented video at:  ./augmented_data/video/This-test.mp4
Created new augmented video at:  ./augmented_data/video/Much-test.mp4


OpenCV: Couldn't read video stream from file "./Anusha Recordings/Anusha Recordings/.DS_S.mp4"


Created new augmented video at:  ./augmented_data/video/What-test.mp4
Created new augmented video at:  ./augmented_data/video/We-test.mp4
Created new augmented video at:  ./augmented_data/video/Is-test.mp4
Created new augmented video at:  ./augmented_data/video/Privacy-test.mp4
Created new augmented video at:  ./augmented_data/video/When-test.mp4
Created new augmented video at:  ./augmented_data/video/Laura-test.mp4
Created new augmented video at:  ./augmented_data/video/Project-test.mp4
Created new augmented video at:  ./augmented_data/video/Been-test.mp4
Created new augmented video at:  ./augmented_data/video/Anusha-test.mp4
Created new augmented video at:  ./augmented_data/video/Hi-test.mp4
Created new augmented video at:  ./augmented_data/video/Have-test.mp4
Created new augmented video at:  ./augmented_data/video/And-test.mp4
Created new augmented video at:  ./augmented_data/video/Where-test.mp4
Created new augmented video at:  ./augmented_data/video/Recording-test.mp4
Created new 

In [20]:
# PCAP Augmnetation
# ideas: 
# add diff delays to packets but keep order
# maybe drop some packets (NEED TO DIFFERENTIATE BETWEEN IMPORTANT AND FILLER)
# add some noise packets (could just be random)

def add_rand_delays(input_path, output_path):
    packets = rdpcap(input_path)

    for packet in packets:
        packet.time += random.uniform(-0.01, 0.01) # +- 10ms

    wrpcap(output_path, packets)
    print(f"Augmented delay PCAP saved to: {output_path}")

def drop_rand_packets(input_path, output_path):
    packets = rdpcap(input_path)

    packets = [packet for packet in packets if not (random.random() < 0.05 and TCP not in packet)]

    wrpcap(output_path, packets)
    print(f"Augmented drop PCAP saved to: {output_path}")

In [ ]:
# PCAP Testing
# tbh idk how we can do testing for this since everything is encrypted lol

In [ ]:
# loop through all possible captures
PACKET_INPUT_PATH_PREFIX = './data_new/'
PACKET_OUTPUT_PATH_PREFIX = './augmented_data/pcap/'
PACKET_SUFFIX = '.pcap'

packet_paths = os.listdir(PACKET_INPUT_PATH_PREFIX)
# print(packet_paths)
for p in packet_paths:
    p = p[:-5]

    # processing delays
    add_rand_delays(PACKET_INPUT_PATH_PREFIX+p+PACKET_SUFFIX, 
                    PACKET_OUTPUT_PATH_PREFIX+p+'-delays'+PACKET_SUFFIX)

    # processing drops
    drop_rand_packets(PACKET_INPUT_PATH_PREFIX+p+PACKET_SUFFIX, 
                      PACKET_OUTPUT_PATH_PREFIX+p+'-drops'+PACKET_SUFFIX)

Augmented delay PCAP saved to: ./augmented_data/pcap/hey-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/hey-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/how-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/how-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/morning-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/morning-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/we-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/we-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/i-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/i-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/technique-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/technique-drops.pcap
Augmented delay PCAP saved to: ./augmented_data/pcap/you-delays.pcap
Augmented drop PCAP saved to: ./augmented_data/pcap/you-drops.pcap
Augmented delay PCAP saved to: ./a

In [36]:
# Augment Both Video and Packets
# mostly syncing for speed changes
def interpolate_frames(frames, speed):
    n_original = len(frames)
    new_indices = np.clip(np.arange(0, n_original, speed), 0, n_original-1).astype(int)
    
    # Select frames
    return [frames[i] for i in new_indices]

def speed_up_augmentation(input_video, output_video, input_packet, output_packet, speed=1.1, fps=24):
    video_frames = video_to_frames(input_video)
    new_video_frames = interpolate_frames(video_frames, speed)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # print(new_video_frames[0])
    frame_height = len(new_video_frames[0])
    frame_width = len(new_video_frames[0][0])
    out = cv2.VideoWriter(output_video, fourcc, 25, (frame_width, frame_height))
    
    for f in new_video_frames:
        out.write(f)

    out.release()
    cv2.destroyAllWindows()
    print('Created new augmented video at: ', output_video)


    packets = rdpcap(input_packet)
    for packet in packets:
        packet.time /= speed
    wrpcap(output_packet, packets)
    print(f"Augmented sync PCAP saved to: {output_packet}")



In [41]:
# Testing sped up video
speed_up_augmentation(VIDEO_INPUT_PATH_PREFIX+'A'+VIDEO_SUFFIX,
                      VIDEO_OUTPUT_PATH_PREFIX+'A'+'-speed2'+VIDEO_SUFFIX,
                      PACKET_INPUT_PATH_PREFIX+'A'+PACKET_SUFFIX,
                      PACKET_OUTPUT_PATH_PREFIX+'A'+'-speed2'+PACKET_SUFFIX, speed=2)

Created new augmented video at:  ./augmented_data/video/A-speed2.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/A-speed2.pcap


In [46]:
# loop through all paths
chosen_speed = 1.1

file_names = os.listdir(VIDEO_INPUT_PATH_PREFIX)
for f in file_names:
    f = f[:-4]

    try:
        speed_up_augmentation(VIDEO_INPUT_PATH_PREFIX+f+VIDEO_SUFFIX,
                        VIDEO_OUTPUT_PATH_PREFIX+f+'-speed'+str(chosen_speed)+VIDEO_SUFFIX,
                        PACKET_INPUT_PATH_PREFIX+f.lower()+PACKET_SUFFIX,
                        PACKET_OUTPUT_PATH_PREFIX+f.lower()+'-speed'+str(chosen_speed)+PACKET_SUFFIX, 
                        speed=chosen_speed)
    except:
        continue
    

Created new augmented video at:  ./augmented_data/video/Which-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/which-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/How-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/how-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/A-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/a-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/Am-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/am-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/My-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/my-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/Teammates-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/teammates-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/291c-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/291c-spe

OpenCV: Couldn't read video stream from file "./Anusha Recordings/Anusha Recordings/.DS_S.mp4"


Created new augmented video at:  ./augmented_data/video/Are-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/are-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/This-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/this-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/Much-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/much-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/What-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/what-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/We-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/we-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/Is-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/is-speed1.1.pcap
Created new augmented video at:  ./augmented_data/video/Privacy-speed1.1.mp4
Augmented sync PCAP saved to: ./augmented_data/pcap/privacy-spe